In [1]:
import os
import pandas as pd
import shutil
import re
import pprint
import math
import numpy as np

# STATIC INPUTS

In [2]:
#import Building Meta Data
buildingMetaData_df = pd.read_excel('Building_MetaData.xlsx')
buildingMetaData_df.head()

,CAAN_Number,Building Name,Building Category,Area (GSF),Currently on District System (Y/N),Existing Building (Y/N),First Year (yr),Last Year (yr),First Year on loop
0,8000,Banana,Academic,65000,Y,Y,1900,3000,2024
1,8001,Apple,Admin,85000,N,Y,1900,3000,2050
2,8002,Orange,Academic,20000,Y,Y,1900,3000,2024


In [3]:
# #import building usage data
# buildingUsage_df = pd.read_excel('Building usage data.xlsx')
# # Sum the loads
# totalSpaceHeating = buildingUsage_df['Heating Load (Btu/h)'].sum()
# totalDHW = buildingUsage_df['DHW Load (Btu/h)'].sum()
# totalHeating = buildingUsage_df['Total Heating Load (Btu/h)'].sum()
# totalLoad = buildingUsage_df['Total Load (Btu/h)'].sum()
# buildingUsage_df.head()



# Import inputs from inputs.py

In [4]:
from ucsbdistrict.inputs import CUP_inputs,building_inputs,building_inputs_df,hotWater_schedule3_inputs_df,domesticWater_schedule4_inputs_df,chilledCTOcean_schedule5_inputs_df, weather_df,buildingModule_inputs,districtModule_inputs,hotWaterMonths

inputs 293738.9131106457


# Building module outputs

### To be looped through every building

In [5]:
#defining inputs ---> static-same for all buildings
HW_LoopSTP=districtModule_inputs["Loop HW STP (°F)"]
HW_supplyLosses=CUP_inputs["Setpoints"]["Hot Water Supply Losses"]
CHW_LoopSTP=districtModule_inputs["Loop CHW STP (°F)"]
CHW_supplyLosses=CUP_inputs["Setpoints"]["Chilled Water Supply Losses"]
CHW_deltaT_Max=building_inputs["Chilled Water"]["Delta T @ Max Load"]
CHW_deltaT_Min=building_inputs["Chilled Water"]["Delta T @ Min Load"]
# CHW_maxLoad= building_inputs["Chilled Water"]["Max Load"]
HHW_BldgSTP= buildingModule_inputs["Building HHW STP (°F)"]
HHW_supply_Temps = building_inputs_df.loc[building_inputs_df["Building Level Categories"]=="Return HHW Temps given Supply Temp","Building Level User Inputs"]
HHW_return_Temps = building_inputs_df.loc[building_inputs_df["Building Level Categories"]=="Return HHW Temps given Supply Temp","Building Level Values"]
DHWMonths = domesticWater_schedule4_inputs_df["Month"]
DHWSetpoint = domesticWater_schedule4_inputs_df["Building"]
# DHWmaxLoad = building_inputs["Domestic Hot Water"]["Max Load"]
DHWmaxApproach = building_inputs["Domestic Hot Water"]["Max Approach"]
DHWminApproach = building_inputs["Domestic Hot Water"]["Min Approach"]
# DHWloadMinApproach = building_inputs["Domestic Hot Water"]["Load @ Min Approach"]

In [6]:
# Reset the index to be sequential (0, 1, 2, 3, ...)
HHW_return_Temps = HHW_return_Temps.reset_index(drop=True)
# data type set to int
HHW_supply_Temps = HHW_supply_Temps.astype(int)

In [7]:

from ucsbdistrict.building_staticInputs import BuildingParameters
#creating instance by assigning static parameters
params_bldg = BuildingParameters(HW_LoopSTP=HW_LoopSTP, HW_supplyLosses=HW_supplyLosses,CHW_LoopSTP=CHW_LoopSTP,\
                                  CHW_supplyLosses=CHW_supplyLosses,CHW_deltaT_Max=CHW_deltaT_Max, CHW_deltaT_Min=CHW_deltaT_Min,\
                                  HHW_supply_Temps=HHW_supply_Temps,HHW_return_Temps=HHW_return_Temps,HHW_BldgSTP=HHW_BldgSTP,\
                                  DHWMonths=DHWMonths,DHWSetpoint=DHWSetpoint,buildingDate=weather_df['Date'],hotWaterMonths=hotWaterMonths,\
                                  DHWmaxApproach=DHWmaxApproach,DHWminApproach=DHWminApproach)


In [8]:
#filtered district building meta data
districtMetaData_df = buildingMetaData_df[buildingMetaData_df["Currently on District System (Y/N)"]=="Y"]

districtMetaData_df.head()

,CAAN_Number,Building Name,Building Category,Area (GSF),Currently on District System (Y/N),Existing Building (Y/N),First Year (yr),Last Year (yr),First Year on loop
0,8000,Banana,Academic,65000,Y,Y,1900,3000,2024
2,8002,Orange,Academic,20000,Y,Y,1900,3000,2024


In [9]:
from ucsbdistrict.building_module import Building
#path to folder that contains all building calibrated data files
building_usage_folder =  r"C:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\Building usage data"
# Get a list of all files in the folder
files = os.listdir(building_usage_folder)
allBldgModule_output =pd.DataFrame()
#loop through all files
for file in files:
    print(file)
    file_path = os.path.join(building_usage_folder, file)
    buildingMockUsage_df = pd.read_excel(file_path)
    if buildingMockUsage_df['CAAN_Number'][0] in districtMetaData_df['CAAN_Number'].values:

        #building Chilled water
        CHW_maxLoad = buildingMockUsage_df['Cooling Load (Btu/h)'].max()
        

        #building DHW
        DHW_maxLoad = buildingMockUsage_df['DHW Load (Btu/h)'].max()
        DHW_loadMinApproach= DHW_maxLoad*0.26

        # #building HHW
        # maxValue_HHW = buildingMockUsage_df['Heating Load (Btu/h)'].max()
        # HHW_maxLoad= maxValue_HHW + DHW_maxLoad

        # creating building instance by  assinging changing / calculated parameters(fields) and calculating outputs
        bldg = Building(parameters=params_bldg, caan_no=buildingMockUsage_df['CAAN_Number'][0], name="Lab",coolingLoad = buildingMockUsage_df['Cooling Load (Btu/h)'],\
                        heatingLoad =buildingMockUsage_df['Heating Load (Btu/h)'],DHWLoad =buildingMockUsage_df['DHW Load (Btu/h)'],\
                        timeStamp = buildingMockUsage_df['Date and Time'],CHW_maxLoad=CHW_maxLoad,DHW_maxLoad=DHW_maxLoad,DHW_loadMinApproach=DHW_loadMinApproach)
        buildingModule_output = bldg.compute()
        allBldgModule_output = pd.concat([allBldgModule_output, buildingModule_output], ignore_index=True)  # Concatenate new row to existing DataFrame

allBldgModule_output.head(10)

8000.xlsx
8001.xlsx
8002.xlsx


,caan_no,Time Stamp,Space Heating Load (Btu/h),DHW Load (Btu/h),Cooling Load (Btu/h),Loop HWST @ Building (°F),Loop CHWST @ Building (°F),CHWRT (°F),CHWR Flow (gpm),Building HHWRT (°F),HHWRflow,Building Domestic Water Temp (°F),Building DHWRT (°F),Building DHWR Flow (gpm),District HWS Flow (gpm),Bypassed Return to HHWS (gpm),District HWR Flow (gpm),District HWRT (°F),Check Building HWS = HWR
0,8000,2010-01-01 00:00:00,6.158529e+06,1.913677e+06,4.892904e+01,165,46.0,54.000009,0.012232,110,615.852893,55,60.176443,36.512341,260.458848,391.906386,260.458848,103.0155,True
1,8000,2010-01-01 01:00:00,4.513664e+06,1.618239e+06,3.492853e+02,165,46.0,54.000061,0.087321,110,451.366397,55,60.000000,30.823601,194.956836,287.233162,194.956836,102.094763,True
2,8000,2010-01-01 02:00:00,5.149001e+06,1.734384e+06,3.492853e+02,165,46.0,54.000061,0.087321,110,514.90006,55,60.000000,33.035878,220.272264,327.663675,220.272264,102.501126,True
3,8000,2010-01-01 03:00:00,5.668696e+06,1.582614e+06,3.492853e+02,165,46.0,54.000061,0.087321,110,566.869554,55,60.000000,30.145019,236.279403,360.735171,236.279403,103.620896,True
4,8000,2010-01-01 04:00:00,5.888233e+06,2.090599e+06,3.492853e+02,165,46.0,54.000061,0.087321,110,588.82329,55,60.679999,40.080497,254.198057,374.70573,254.198057,102.223504,True
5,8000,2010-01-01 05:00:00,5.112278e+06,2.764295e+06,3.982144e+02,165,46.0,54.000070,0.099553,110,511.227764,55,62.597476,53.988803,239.889808,325.326759,239.889808,99.331745,True
6,8000,2010-01-01 06:00:00,4.754129e+06,4.051484e+06,4.471434e+02,165,46.0,54.000078,0.111785,110,475.412885,55,66.261078,82.064580,254.941993,302.535472,254.941993,95.920655,True
7,8000,2010-01-01 07:00:00,4.163426e+06,5.224267e+06,4.960724e+02,165,46.0,54.000087,0.124017,110,416.342578,55,69.599058,109.522346,260.919647,264.945277,260.919647,93.0415,True
8,8000,2010-01-01 08:00:00,3.642689e+06,5.962181e+06,1.467871e+02,165,46.0,54.000026,0.036697,110,364.268861,55,71.699312,127.805731,260.267135,231.807457,260.267135,91.192218,True
9,8000,2010-01-01 09:00:00,5.462850e+06,6.587429e+06,1.079918e+06,165,46.0,54.189233,263.740986,110,546.284975,55,73.478893,143.954306,342.603388,347.635893,342.603388,94.654645,True


In [10]:
# Convert 'Time Stamp' column to datetime format
allBldgModule_output['Time Stamp'] = pd.to_datetime(allBldgModule_output['Time Stamp'])

# Group by 'Time Stamp' and sum the 'Loop HWST @ Building (°F)' column
sum_by_timestamp = allBldgModule_output.groupby('Time Stamp')['Loop HWST @ Building (°F)'].sum()
print(sum_by_timestamp.shape)

(8760,)


In [11]:
print(8760*2)

17520


In [12]:
# from ucsbdistrict.building_module import Building
# #creating building instance bu  assinging changing / calculated parameters(fields) and calculating outputs
# bldg_1 = Building(parameters=params_bldg, caan_no=1000, name="Lab",coolingLoad = buildingUsage_df['Cooling Load (Btu/h)'],\
#                  heatingLoad =buildingUsage_df['Heating Load (Btu/h)'],DHWLoad =buildingUsage_df['DHW Load (Btu/h)'],\
#                  timeStamp = buildingMockUsage_df['Date and Time'])
# buildingModule_output = bldg_1.compute()
# buildingModule_output.head(15)

In [13]:
from ucsbdistrict.building_mixing import building_mixing

bldgs_mixed = building_mixing(df=allBldgModule_output,parameters=params_bldg)
buildingMixing_output = bldgs_mixed.compute()
buildingMixing_output.head()

,Total Space Heating Load (Btu/h),Total DHW Load (Btu/h),Total Heating Load (Btu/h),District HWS Flow (gpm),District HWRT (°F),District CHWS Flow (gpm),District CHWRT (°F),Total Cooling Load (Btu/h)
0,1.231706e+07,3.827353e+06,1.614441e+07,520.917695,103.0155,0.024464,54.000009,122.322571
1,9.027328e+06,3.236478e+06,1.226381e+07,389.913673,102.094763,0.174641,54.000061,873.211983
2,1.029800e+07,3.468767e+06,1.376677e+07,440.544528,102.501126,0.174641,54.000061,873.211983
3,1.133739e+07,3.165227e+06,1.450262e+07,472.558806,103.620896,0.174641,54.000061,873.211983
4,1.177647e+07,4.181198e+06,1.595766e+07,508.396115,102.223504,0.174641,54.000061,873.211983


In [14]:
buildingMixing_output.to_pickle('buildingMixing_output.pkl')